In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9212259107649648837,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14349385063696313470
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14694015851225181755
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3615640280207632478
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.4.3
tensorflow Version 2.3.0


In [5]:
batch_size = 32
img_height, img_width = 800, 600
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
print(os.listdir("../input/data-pig/"))

['train', 'test']


In [7]:
train_dir = '../input/data-pig/train/'
test_dir = '../input/data-pig/test/'

In [8]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [9]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect',
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 8061 images belonging to 4 classes.
Found 2012 images belonging to 4 classes.
Found 1776 images belonging to 4 classes.


In [10]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 8061
nb_validation_samples: 2012
nb_test_samples: 1776

predict_size_train: 252
predict_size_validation: 63
predict_size_test: 56

 num_classes: 4


In [11]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "VGG19_descriptors"

In [12]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [13]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [14]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionResNetV2(weights=inception_resnet_v2_weights, include_top=False, pooling = "avg")

In [15]:
# base_model1.summary()

In [16]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [17]:
bottleneck_final_model = base_model1

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [21]:

# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [22]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [23]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        'hidden1':hp.choice('hidden1',[4096,2048,1024,512,256]),
#         'hidden2':hp.choice('hidden2',[2048,1024,512,256,128,]),
#         'hidden3':hp.choice('hidden3',[1024,512,256,128,64,32]),    
        'bias_reg': hp.choice('bias_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'act_reg': hp.choice('act_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'ker_reg': hp.choice('ker_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'activation_function':hp.choice('activation_function',["relu","elu","selu","softplus","tanh",])
        }

In [24]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
#     dropout_rate = 0.5
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(params["hidden1"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
#     model.add(Dropout(0.25))

#     model.add(Dense(params["hidden2"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
#     model.add(Dense(params["hidden3"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
    model.add(Dropout(0.5))
    

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [25]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [26]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:                                    
{'act_reg': 1e-05, 'activation_function': 'relu', 'beta_1': 0.4, 'beta_2': 0.4, 'bias_reg': 0.0001, 'hidden1': 256, 'ker_reg': 0.01, 'lr': 0.001}
Epoch 1/1000                                           
252/252 - 1s - loss: 1.6157 - accuracy: 0.6427 - val_loss: 1.1776 - val_accuracy: 0.5835

Epoch 2/1000                                           
252/252 - 1s - loss: 0.7500 - accuracy: 0.8155 - val_loss: 0.7455 - val_accuracy: 0.8141

Epoch 3/1000                                           
252/252 - 1s - loss: 0.5920 - accuracy: 0.8642 - val_loss: 0.5524 - val_accuracy: 0.9100

Epoch 4/1000                                           
252/252 - 1s - loss: 0.5292 - accuracy: 0.8773 - val_loss: 0.7448 - val_accuracy: 0.7450

Epoch 5/1000                                           
252/252 - 1s - loss: 0.4838 - accuracy: 0.8877 - val_loss: 0.4049 - val_accuracy: 0.9543

Epoch 6/1000                                           
252/252 - 1s

252/252 - 1s - loss: 0.4467 - accuracy: 0.8521 - val_loss: 0.2282 - val_accuracy: 0.9677

Epoch 56/1000                                          
252/252 - 1s - loss: 0.4391 - accuracy: 0.8526 - val_loss: 0.2207 - val_accuracy: 0.9662

Epoch 57/1000                                          
252/252 - 1s - loss: 0.4555 - accuracy: 0.8504 - val_loss: 0.4102 - val_accuracy: 0.8767

Epoch 58/1000                                          
252/252 - 1s - loss: 0.4634 - accuracy: 0.8436 - val_loss: 0.3007 - val_accuracy: 0.9230

Epoch 59/1000                                          
252/252 - 1s - loss: 0.4713 - accuracy: 0.8407 - val_loss: 0.2288 - val_accuracy: 0.9597

Epoch 60/1000                                          
252/252 - 1s - loss: 0.4548 - accuracy: 0.8489 - val_loss: 0.2332 - val_accuracy: 0.9642

Epoch 61/1000                                          
252/252 - 1s - loss: 0.4865 - accuracy: 0.8267 - val_loss: 0.2714 - val_accuracy: 0.9329

Epoch 62/1000                     

252/252 - 1s - loss: 0.5255 - accuracy: 0.8460 - val_loss: 0.2792 - val_accuracy: 0.9488

Epoch 112/1000                                         
252/252 - 1s - loss: 0.5526 - accuracy: 0.8331 - val_loss: 0.2829 - val_accuracy: 0.9458

Epoch 113/1000                                         
252/252 - 1s - loss: 0.5191 - accuracy: 0.8389 - val_loss: 0.4234 - val_accuracy: 0.8633

Epoch 114/1000                                         
252/252 - 1s - loss: 0.5157 - accuracy: 0.8432 - val_loss: 0.3383 - val_accuracy: 0.9145

Epoch 115/1000                                         
252/252 - 1s - loss: 0.5259 - accuracy: 0.8405 - val_loss: 0.4551 - val_accuracy: 0.8653

Epoch 116/1000                                         
252/252 - 1s - loss: 0.5358 - accuracy: 0.8325 - val_loss: 0.3644 - val_accuracy: 0.9433

Epoch 117/1000                                         
252/252 - 1s - loss: 0.5561 - accuracy: 0.8096 - val_loss: 0.3568 - val_accuracy: 0.8951

Epoch 118/1000                    

252/252 - 1s - loss: 0.6419 - accuracy: 0.7637 - val_loss: 0.4450 - val_accuracy: 0.9021

Epoch 168/1000                                         
252/252 - 1s - loss: 0.6274 - accuracy: 0.7736 - val_loss: 0.4920 - val_accuracy: 0.7808

Epoch 169/1000                                         
252/252 - 1s - loss: 0.6259 - accuracy: 0.7767 - val_loss: 0.4305 - val_accuracy: 0.8628

Epoch 170/1000                                         
252/252 - 1s - loss: 0.6228 - accuracy: 0.7762 - val_loss: 0.2999 - val_accuracy: 0.9493

Epoch 171/1000                                         
252/252 - 1s - loss: 0.6190 - accuracy: 0.7797 - val_loss: 0.3257 - val_accuracy: 0.9215

Epoch 172/1000                                         
252/252 - 1s - loss: 0.6262 - accuracy: 0.7822 - val_loss: 0.4547 - val_accuracy: 0.8469

Epoch 173/1000                                         
252/252 - 1s - loss: 0.6329 - accuracy: 0.7836 - val_loss: 0.4908 - val_accuracy: 0.8320

Epoch 174/1000                    

252/252 - 1s - loss: 0.6207 - accuracy: 0.7863 - val_loss: 0.4087 - val_accuracy: 0.9001

Epoch 224/1000                                         
252/252 - 1s - loss: 0.6124 - accuracy: 0.7910 - val_loss: 0.3703 - val_accuracy: 0.9543

Epoch 225/1000                                         
252/252 - 1s - loss: 0.6317 - accuracy: 0.7848 - val_loss: 0.4011 - val_accuracy: 0.9289

Epoch 226/1000                                         
252/252 - 1s - loss: 0.6531 - accuracy: 0.7791 - val_loss: 0.4806 - val_accuracy: 0.9359

Epoch 227/1000                                         
252/252 - 1s - loss: 0.6316 - accuracy: 0.7920 - val_loss: 0.3159 - val_accuracy: 0.9662

Epoch 228/1000                                         
252/252 - 1s - loss: 0.6093 - accuracy: 0.7969 - val_loss: 0.2816 - val_accuracy: 0.9528

Epoch 229/1000                                         
252/252 - 1s - loss: 0.6089 - accuracy: 0.7985 - val_loss: 0.5370 - val_accuracy: 0.8275

Epoch 230/1000                    

...

252/252 - 1s - loss: 0.2374 - accuracy: 0.9692 - val_loss: 0.6825 - val_accuracy: 0.7460

Epoch 58/1000                                                                        
252/252 - 1s - loss: 0.2400 - accuracy: 0.9670 - val_loss: 0.3555 - val_accuracy: 0.9379

Epoch 59/1000                                                                        
252/252 - 1s - loss: 0.2350 - accuracy: 0.9684 - val_loss: 0.3460 - val_accuracy: 0.9245

Epoch 60/1000                                                                        
252/252 - 1s - loss: 0.2328 - accuracy: 0.9715 - val_loss: 0.3776 - val_accuracy: 0.8996

Epoch 61/1000                                                                        
252/252 - 1s - loss: 0.2350 - accuracy: 0.9687 - val_loss: 0.2802 - val_accuracy: 0.9587

Epoch 62/1000                                                                        
252/252 - 1s - loss: 0.2414 - accuracy: 0.9670 - val_loss: 0.2854 - val_accuracy: 0.9578

Epoch 63/1000                 

Epoch 104/1000                                                                       
252/252 - 1s - loss: 0.2274 - accuracy: 0.9634 - val_loss: 0.5188 - val_accuracy: 0.8290

Epoch 105/1000                                                                       
252/252 - 1s - loss: 0.2161 - accuracy: 0.9684 - val_loss: 0.2277 - val_accuracy: 0.9717

Epoch 106/1000                                                                       
252/252 - 1s - loss: 0.2227 - accuracy: 0.9635 - val_loss: 0.2271 - val_accuracy: 0.9737

Epoch 107/1000                                                                       
252/252 - 1s - loss: 0.2276 - accuracy: 0.9630 - val_loss: 0.3710 - val_accuracy: 0.9061

Epoch 108/1000                                                                       
252/252 - 1s - loss: 0.2150 - accuracy: 0.9670 - val_loss: 0.5713 - val_accuracy: 0.8057

Epoch 109/1000                                                                       
252/252 - 1s - loss: 0.2239 - accu

252/252 - 1s - loss: 0.2179 - accuracy: 0.9627 - val_loss: 0.5681 - val_accuracy: 0.8121

Epoch 151/1000                                                                       
252/252 - 1s - loss: 0.2142 - accuracy: 0.9676 - val_loss: 0.2293 - val_accuracy: 0.9687

Epoch 152/1000                                                                       
252/252 - 1s - loss: 0.2183 - accuracy: 0.9628 - val_loss: 0.2350 - val_accuracy: 0.9697

Epoch 153/1000                                                                       
252/252 - 1s - loss: 0.2105 - accuracy: 0.9707 - val_loss: 0.6252 - val_accuracy: 0.7828

Epoch 154/1000                                                                       
252/252 - 1s - loss: 0.2164 - accuracy: 0.9634 - val_loss: 0.3126 - val_accuracy: 0.9289

Epoch 155/1000                                                                       
252/252 - 1s - loss: 0.2107 - accuracy: 0.9664 - val_loss: 0.2194 - val_accuracy: 0.9722

Epoch 156/1000                

Epoch 197/1000                                                                       
252/252 - 1s - loss: 0.2099 - accuracy: 0.9641 - val_loss: 0.3498 - val_accuracy: 0.9021

Epoch 198/1000                                                                       
252/252 - 1s - loss: 0.2056 - accuracy: 0.9675 - val_loss: 0.5345 - val_accuracy: 0.8255

Epoch 199/1000                                                                       
252/252 - 1s - loss: 0.2118 - accuracy: 0.9656 - val_loss: 0.4684 - val_accuracy: 0.8743

Epoch 200/1000                                                                       
252/252 - 1s - loss: 0.2090 - accuracy: 0.9659 - val_loss: 0.2898 - val_accuracy: 0.9453

Epoch 201/1000                                                                       
252/252 - 1s - loss: 0.2154 - accuracy: 0.9646 - val_loss: 0.3938 - val_accuracy: 0.8822

Epoch 202/1000                                                                       
252/252 - 1s - loss: 0.2008 - accu

KeyboardInterrupt: 

In [ ]:
print("Done!!!!!!!!!!!!!!!!!!!!!!!!")